In [26]:
import time

"""
Sudoku Solver by Evan Freeman
Basic Technique, Then Brute Force


Input must be a string of 81 characters
Blank cells may be filled with anything for the input.



For example:
    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8
    
Which solves to:
    794582136268931745315476982689715324432869571157243869821657493943128657576394218


Here's the coordinates of every cell in the grid:

(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)
(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)
(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)
(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)
(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)
(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)
(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)
(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)
(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)


Big picture:
    1) Start with naked / hidden singles techniques. This is enough for many puzzle.
        -Use the "little numbers" technique, where each cell has a "bag" of possible numbers that gets updated.
        -We will need to store these numbers in a list, so we can iterate through them as we 
    2) When you get stuck, finish off with brute force, but only considering 

To do:
    1) Accept more input types
    2) Check for multiple solutions
    3) Implement sudoku strategies to speed up solve time:
        -Could still finish off with brute force, even if I just put in a few basic strategies
        -Do the 'little numbers' technique, where I keep track of the possibilites associated with each cell
    4) Make it display as it goes?
        -That means showing MILLIONS of iterations. Would have to animate SUPER FAST.
        -Or collect all those images and make a gif of it, after the fact.
            - Pre-complile??? I don't know what I'm talking about
    5) Just how brute force is my algorithm? I think it's slightly faster than just populating every blank with a guess,
        checking the whole puzzle, then updating the whole puzzle and checking again (or even just checking the parts effected
        by the update).
        -After all, my algorithm cuts off certian possibility spaces as it goes
        -Still pretty brute force
    6) YES, I SHOULD HAVE MADE THE BLANKS OBJECTS, MAYBE EVEN ASSOCIATED THEM WITH THE CELL OBJECT SOMEHOW. SUE ME!!!
        -also should have named them "blanks" instead of "b"
        
        
Thoughts:
    Brute force time to complete depends on a few things:
        1) Luck. Whether a given blank cell will end up being 1 or 9 makes a big difference, as we start at 1.
        2) Contradiction depth. How long, on average, we can randomly fill cells before creating a contradiction?

"""

'\nSudoku Solver by Evan Freeman\nBasic Technique, Then Brute Force\n\n\nInput must be a string of 81 characters\nBlank cells may be filled with anything for the input.\n\n\n\nFor example:\n    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8\n    \nWhich solves to:\n    794582136268931745315476982689715324432869571157243869821657493943128657576394218\n\n\nHere\'s the coordinates of every cell in the grid:\n\n(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)\n(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)\n(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)\n(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)\n(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)\n(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)\n(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)\n(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)\n(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)\n\n\nBig picture:\n    1) Start with naked / hidden singles 

In [27]:
#Here is my original, pretty brute force, method
def solve_bf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, in the format of ['.', i, j]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                #so keep track of the cell we are going to fill, and it's coordinates
                b.append([sudoku.cells[i][j], i, j])

    #Initialize some variables
    i = 0    
    count = 0 

    #This is the engine that drives the solution 
    #In each scenario, we update both the list of blanks, and the sudoku grid itself
    #Keep going until our index hits the length of blanks (Which is to say, we're one step beyond)
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with 1
        if b[i][0] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            b[i][0] = '1'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at 9. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == '9':
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some number 1-8 already plugged in. So we step forward by one.
        else:
            b[i][0] = str(int(b[i][0]) + 1)
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1

    #Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

In [28]:
#Here is my less brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9

def solve_lbf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the less brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                b.append([sudoku.cells[i][j], i, j, real_poss])
    
    
    
    #Step 2: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility
    
    i = 0
    count = 0
    
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if b[i][0] == '.':
            b[i][0] = b[i][3][0]
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == b[i][3][-1]:
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            b[i][0] = b[i][3][b[i][3].index(b[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

In [29]:
#Here is my updated solver
#It uses only hidden single and naked single, then finishes with brute force if necessary

def solve(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the mostly not brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                b.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in b:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(b)):
            if len(b[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[b[i][1]][b[i][2]] = b[i][3][0]
                # Delete that entry in the blanks
                del b[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in b:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in b if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in b if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same remainder when divided by 3
            blank_box = [other_blank for other_blank in b if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    b.remove(blank)
                    return True
        return False
            
            
        
                
                
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
    
    ns_count = 0
    hs_count = 0
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()
                       
        prog2 = hidden_single()
        update_blanks()
        
        ns_count += prog1
        hs_count += prog2
        progress = prog1 or prog2
    
    sudoku.display()
    print(f'We solved {ns_count} cells with naked singles.')
    print(f'We solved {hs_count} cells with hidden singles.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if b[i][0] == '.':
            b[i][0] = b[i][3][0]
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == b[i][3][-1]:
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            b[i][0] = b[i][3][b[i][3].index(b[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'Brute force: {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('-'*200)
    print('\n')
    return solution

In [30]:
# I've implemented some particular puzzles for it to solve.
# You can uncomment the lines below to make it ask for an input,or just plug in your own puzzle below.

# puzzle = input('Please input your sudoku puzzle as a string of 81 characters. ')      
# solve(puzzle)

In [31]:
#Random Sudoku

solve_bf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve_lbf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')

Here is the brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- This program took 11.304571866989136 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 1219898 loops.


--- This program took 5.8

'794582136268931745315476982689715324432869571157243869821657493943128657576394218'

In [32]:
# Easy Puzzle 5,220,548,762 from Web Sudoku

solve_bf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve_lbf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')

Here is the brute force solution result:

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547
352467918

243976185685132479197854362568243791721695834439781256974518623816329547352467918


This sudoku was solved in 2773 loops.


--- This program took 0.02099299430847168 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547
352467918

243976185685132479197854362568243791721695834439781256974518623816329547352467918


This sudoku was solved in 926 loops.


--- This program took 0.0100877

'243976185685132479197854362568243791721695834439781256974518623816329547352467918'

In [33]:
#Some easy one, forgot where this came from

solve_bf('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')
solve_lbf('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')
solve('............942.8.16.....29........89.6.....14..25......4.......2...8.9..5....7..')

Here is the brute force solution result:

.........
...942.8.
16.....29
........8
9.6.....1
4..25....
..4......
.2...8.9.
.5....7..


249186573
735942186
168375429
512697348
976834251
483251967
694723815
327518694
851469732

249186573735942186168375429512697348976834251483251967694723815327518694851469732


This sudoku was solved in 317144 loops.


--- This program took 1.543741226196289 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.........
...942.8.
16.....29
........8
9.6.....1
4..25....
..4......
.2...8.9.
.5....7..


249186573
735942186
168375429
512697348
976834251
483251967
694723815
327518694
851469732

249186573735942186168375429512697348976834251483251967694723815327518694851469732


This sudoku was solved in 170446 loops.


--- This program took 0.7648

'249186573735942186168375429512697348976834251483251967694723815327518694851469732'

In [34]:
#Kindof Hard 1

solve_bf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve_lbf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')

Here is the brute force solution result:

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 54770833 loops.


--- This program took 249.13443994522095 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 23017404 loops.


--- This program took 1

'653287941794631258128945376819724563236859417547163829965372184372418695481596732'

In [35]:
#More of a medium

solve_bf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve_lbf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')

Here is the brute force solution result:

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746
645791328

152479683368215974974638512416387259783952461529146837237864195891523746645791328


This sudoku was solved in 3227386 loops.


--- This program took 15.669791221618652 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746
645791328

152479683368215974974638512416387259783952461529146837237864195891523746645791328


This sudoku was solved in 1600652 loops.


--- This program took 7.6

'152479683368215974974638512416387259783952461529146837237864195891523746645791328'

In [36]:
#Greg's Evil Puzzle

solve_bf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve_lbf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')

Here is the brute force solution result:

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698
936857142

213965874894371265675482931569718423348529716721643589482196357157234698936857142


This sudoku was solved in 98209 loops.


--- This program took 0.46126222610473633 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698
936857142

213965874894371265675482931569718423348529716721643589482196357157234698936857142


This sudoku was solved in 41182 loops.


--- This program took 0.1862

'213965874894371265675482931569718423348529716721643589482196357157234698936857142'

In [37]:
#Easiest Possible Sudoku, from sudoku wiki
#Only requires naked singles

solve_bf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve_lbf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')

Here is the brute force solution result:

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
831459267

672145398145983672389762451263574819958621743714398526597236184426817935831459267


This sudoku was solved in 3115 loops.


--- This program took 0.017749547958374023 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
831459267

672145398145983672389762451263574819958621743714398526597236184426817935831459267


This sudoku was solved in 1190 loops.


--- This program took 0.00789

'672145398145983672389762451263574819958621743714398526597236184426817935831459267'

In [39]:
#Very Hard Benchmark 1

solve_bf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve_lbf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

Here is the brute force solution result:

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 97273649 loops.


--- This program took 495.90692687034607 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 54959559 loops.


--- This program took 2

'417369825632158947958724316825437169791586432346912758289643571573291684164875293'